Analysis of hyperparameter searches

In [11]:
from master_scripts.data_functions import get_git_root
import json
import pandas

In [40]:
def load_experiment(e_id):
    repo_root = get_git_root()
    e_path = repo_root + "experiments/"
    with open(e_path + e_id + ".json", "r") as fp:
        e = json.load(fp)
    return e

def load_hparam_search(name):
    """ Reads scsv-formatted hparam search file spec to pandas DF,
    and loads additional metrics into the dataframe.
    """
    repo_root = get_git_root()
    hpath = repo_root + "experiments/"
    df = pandas.read_csv(hpath + name, sep=';')
    
    # Add additional metrics to df
    accs = []
    f1 = []
    mcc = []
    auc = []
    for e_id in df['id']:
        e = load_experiment(e_id)
        accs.append(e['metrics']['accuracy_score'])
        f1.append(e['metrics']['f1_score'])
        mcc.append(e['metrics']['matthews_corrcoef'])
        auc.append(e['metrics']['roc_auc_score'])
    df['accuracy_score'] = accs
    df['f1_score'] = f1
    df['matthews_corrcoef'] = mcc
    df['roc_auc_score'] = auc
    return df

In [41]:
df = load_hparam_search("architecture_search.scsv")
# Architecture search number 1 has swapped TP and FP

In [42]:
df

,id,filters,kernel_size,dense_size,accuracy_score,f1_score,matthews_corrcoef,roc_auc_score
0,53e267bb150f,4,"(3, 3)",32,0.96116,0.960366,0.923090,0.986292
1,0e52e327b0d5,4,"(3, 3)",64,0.91442,0.918509,0.832979,0.978133
2,3fe459e5b200,4,"(3, 3)",128,0.92530,0.925146,0.850610,0.978812
3,2e5e1e2a7809,4,"(3, 3)",256,0.94446,0.945411,0.889437,0.986247
4,21f0e7a8fb10,4,"(5, 5)",32,0.95016,0.949711,0.900476,0.985682
...,...,...,...,...,...,...,...,...
75,3e12a1e318cd,64,"(7, 7)",256,0.94206,0.943284,0.884916,0.986605
76,354c8adda0a8,64,"(9, 9)",32,0.91974,0.924279,0.845498,0.986435
77,9a820dac9216,64,"(9, 9)",64,0.93698,0.937851,0.874285,0.985109
78,45580d76242c,64,"(9, 9)",128,0.92650,0.930185,0.857722,0.988957
